# Mon 1er Réseau Neuronal

<img src="https://www.tibco.com/sites/tibco/files/media_entity/2021-05/neutral-network-diagram.svg">

## Description: 
- Choisir un Dataset simple et clean. 
- Choisir de faire une régression ou classification. 
- Construire le réseau neuronal avec l'exemple suivant. 
- Les hidden layers pourrons avoir des fonctions d'activation "relu".

## Rendu: 
- Présentation schematique du réseau neuronal
- Présentation du notebook
    - La construction du réseau
    - Les résultats obtenus
    - Le scoring

In [1]:
# !pip install tensorflow
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.compose import ColumnTransformer, make_column_selector,make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_validate, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn import set_config
set_config(display='diagram')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.keras import *

2023-04-07 11:01:50.985739: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-07 11:01:50.985847: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
from tensorflow.keras import Sequential, layers

In [4]:
# Ex:
# X de 15 features
# y target de 3 dim

In [5]:
### Regression of size 4
model = Sequential()
model.add(layers.Dense(10, activation='relu', input_dim=15))
#model.add(...)
model.add(layers.Dense(3, activation='linear'))


### Classification with 8 classes
model = Sequential()
model.add(layers.Dense(10, activation='relu', input_dim=15))
#model.add(...)
model.add(layers.Dense(3, activation='softmax'))

2023-04-07 11:01:55.224062: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-07 11:01:55.224545: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-07 11:01:55.224936: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DjelarAS): /proc/driver/nvidia/version does not exist
2023-04-07 11:01:55.229064: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# Ex:
# X de 9 features
# y target de 1 dim

In [7]:
### Regression of size 4
model = Sequential()
model.add(layers.Dense(10, activation='relu', input_dim=9))
#model.add(...)
model.add(layers.Dense(1, activation='linear'))


### Classification with 8 classes
model = Sequential()
model.add(layers.Dense(10, activation='relu', input_dim=9))
#model.add(...)
model.add(layers.Dense(1, activation='softmax'))

In [8]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 10)                100       
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                                 
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy', 
    metrics = 'accuracy')

In [10]:
model.fit(X, y, batch_size=16, epochs=10)

NameError: name 'X' is not defined

In [ ]:
model.evaluate(scaler.transform(X), y)

In [ ]:
model.predict(scaler.transform(X))